In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
import os, sys

import pickle
from pathlib import Path
import time
import matplotlib.pyplot as plt


from src.WIP_pdotw_toy import toy_ini_sol, toy_example, toy_best_insertion
from src.pdpt_rasd import select_subroutes, pdpt_route_schedule_decomposition
from src.util import read_pdpt_pickle, read_route_solution_PDPT

VERBOSE = 0

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/toy'


ModuleNotFoundError: No module named 'util'

In [ ]:
def pdpt_rasd(dir_, verbose = 0):

    iniSol_filename = dir_ + '/toyinitSol.txt'
    pdpt_ins = read_pdpt_pickle(dir_ +'/toy.pkl', verbose = verbose-1) 

    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(iniSol_filename, verbose = 0)

    initial_solution = (truck_yCycle_file, truck_used_file, truck_route_file, \
                        cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
                        Sb_sol_file, Ab_sol_file, Db_sol_file)

    subroutes = select_subroutes(pdpt_ins, cargo_route_file, verbose)

    MP_sol, SP_sol, route_sol = pdpt_route_schedule_decomposition(dir_+'/toy', pdpt_ins, initial_solution, subroutes, verbose = 0)

    res = {'MP': {'x_sol': MP_sol[0],
                  'x_sol': MP_sol[1],
                  'z_sol': MP_sol[2],
                  'y_sol': MP_sol[3],
                  'u_sol': MP_sol[4],
                  'D_sol': MP_sol[5],
                 },
            'SP':{'g_sol': SP_sol[0],
                  'h_sol': SP_sol[1],
                  'D_sol': SP_sol[-1],
                 },
            'route':{'truck_route': route_sol[0],
                     'cargo_route': route_sol[-1],
                    },
            }
    
    res_filename = dir_ + '/toyimprove.pkl'
    with open(res_filename, 'wb') as pickle_file:
        pickle.dump(res, pickle_file)


In [ ]:
def plot_instance(dir_, truck_colors, cargo_colors, font):

    iniSol_filename = dir_ + '/toyinitSol.txt'
    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(iniSol_filename, verbose = 0)
    pdpt_ins = read_pdpt_pickle(dir_ +'/toy.pkl', verbose = 0) 

    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']
    cargo_list = pdpt_ins['cargo']
    node_coor = pdpt_ins['loc']

    fig1, ax1 = plt.subplots(1,1, figsize=(10,5))
    ax1.axis('off')
    
    cargo_idx = 0
    for cargo_key, cargo_value in cargo_list.items():
        source, dest = int(cargo_value[-2]), int(cargo_value[-1])
        ax1.plot(*node_coor[source], 'o', color = cargo_colors[cargo_idx],
                 mec='k', ms=10)
        ax1.plot(*node_coor[dest], '^', color = cargo_colors[cargo_idx],
                 mec='k', ms=10)

        cargo_idx+=1
        
    legend_elements = [Line2D([0], [0],  marker='^', color='w', label='origin',
                          markerfacecolor='None', mec='k',  markersize=10),
                   Line2D([0], [0],  marker='o', color='w', label='destination',
                          markerfacecolor='None', mec='k', markersize=10)]
        
    ax1.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=2, prop=font)


    ax1.set_title('Toy Instance', size=18, font='serif')

    return fig1, ax1
        

In [ ]:
def plot_init_Sol(dir_, truck_colors, cargo_colors, font):

    iniSol_filename = dir_ + '/toyinitSol.txt'
    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(iniSol_filename, verbose = 0)
    pdpt_ins = read_pdpt_pickle(dir_ +'/toy.pkl', verbose = 0) 

    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']
    cargo_list = pdpt_ins['cargo']
    node_coor = pdpt_ins['loc']

    fig1, ax1 = plt.subplots(1,1, figsize=(10,5))
    ax1.axis('off')
    
    cargo_idx = 0
    for cargo_key, cargo_value in cargo_list.items():
        source, dest = int(cargo_value[-2]), int(cargo_value[-1])
        ax1.plot(*node_coor[source], 'o', color = cargo_colors[cargo_idx],
                 mec='k', ms=10)
        ax1.plot(*node_coor[dest], '^', color = cargo_colors[cargo_idx],
                 mec='k', ms=10)

        cargo_idx+=1
        
    truck_idx = 0
    for truck_key, route_ in truck_route_file.items():
        for i in range(len(route_)-1):
            node_curr, node_next = int(route_[i]), int(route_[i+1])
            x0, y0 = node_coor[node_curr]
            dx, dy = (np.array(node_coor[node_next])-np.array(node_coor[node_curr]))
            ax1.arrow(x0, y0, dx, dy, color=truck_colors[truck_idx], head_width=.08,
                      length_includes_head = True, linewidth=2, linestyle=':',
                      alpha=0.3+truck_idx*0.1)
        truck_idx += 1
        
    legend_elements = [Line2D([0], [0],  marker='^', color='w', label='origin',
                          markerfacecolor='None', mec='k',  markersize=10),
                       Line2D([0], [0],  marker='o', color='w', label='destination',
                          markerfacecolor='None', mec='k', markersize=10)]
    
    for i in range(len(truck_colors)):
        legend_elements.append(Line2D([0], [0], color=truck_colors[i], 
                             lw=4, alpha=0.3+truck_idx*0.1, label=f'Truck {i+1}'))

    ax1.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=5, prop = font)

    ax1.set_title('Toy Instance Initial Solution from PDOTWs', size=18, font='serif')
    
    return fig1, ax1

In [ ]:
def plot_rasd_sol(dir_, truck_colors, cargo_colors, font):

    iniSol_filename = dir_ + '/toyinitSol.txt'
    truck_yCycle_file, truck_used_file, truck_route_file, \
    cargo_route_file, S_sol_file, A_sol_file, D_sol_file, \
    Sb_sol_file, Ab_sol_file, Db_sol_file = read_route_solution_PDPT(iniSol_filename, verbose = 0)
    pdpt_ins = read_pdpt_pickle(dir_ +'/toy.pkl', verbose = 0) 

    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']
    cargo_list = pdpt_ins['cargo']
    node_coor = pdpt_ins['loc']

    
    filename = dir_ + '/toyimprove.pkl'
    with open(filename, 'rb') as pickle_file:
        rasd_sol=pickle.load(pickle_file)
    
    subroute_truck_route = rasd_sol['route']['truck_route']
    trucks_in_subroute = subroute_truck_route.keys()
    
    fig1, ax1 = plt.subplots(1,1, figsize=(10,5))
#     ax1.plot(*np.array(node_coor).T, 'o',ms =10, mfc='None', mec='k', mew=2)
    ax1.axis('off')
    
    cargo_idx = 0
    for cargo_key, cargo_value in cargo_list.items():
        source, dest = int(cargo_value[-2]), int(cargo_value[-1])
        ax1.plot(*node_coor[source], 'o', color = cargo_colors[cargo_idx],
                 mec='k', ms=10, label='_nolegend_')
        ax1.plot(*node_coor[dest], '^', color = cargo_colors[cargo_idx],
                 mec='k', ms=10, label='_nolegend_')

        cargo_idx+=1

    truck_idx = 0
    for truck_key, route_ in truck_route_file.items():
        if truck_key in trucks_in_subroute:
            route_ = subroute_truck_route[truck_key]
            for i in range(len(route_)-1):
                node_curr, node_next = int(route_[i]), int(route_[i+1])
                x0, y0 = node_coor[node_curr]
                dx, dy = (np.array(node_coor[node_next])-np.array(node_coor[node_curr]))
                ax1.arrow(x0, y0, dx, dy, color=truck_colors[truck_idx], head_width=.08,
                          length_includes_head = True, linewidth=2, linestyle=':',
                          alpha=0.3+truck_idx*0.1)
            truck_idx += 1
        elif truck_key not in trucks_in_subroute:
            for i in range(len(route_)-1):
                node_curr, node_next = int(route_[i]), int(route_[i+1])
                x0, y0 = node_coor[node_curr]
                dx, dy = (np.array(node_coor[node_next])-np.array(node_coor[node_curr]))
                ax1.arrow(x0, y0, dx, dy, color=truck_colors[truck_idx], head_width=.08,
                          length_includes_head = True, linewidth=2, linestyle=':',
                          alpha=0.3+truck_idx*0.1)
            truck_idx += 1
            

        
    legend_elements = [Line2D([0], [0],  marker='^', color='w', label='origin',
                          markerfacecolor='None', mec='k',  markersize=10),
                       Line2D([0], [0],  marker='o', color='w', label='destination',
                          markerfacecolor='None', mec='k', markersize=10)]
    
    for i in range(len(truck_colors)):
        legend_elements.append(Line2D([0], [0], color=truck_colors[i], 
                             lw=4, alpha=0.3+truck_idx*0.1, label=f'Truck {i+1}'))

    ax1.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05),
              fancybox=True, shadow=True, ncol=5, prop = font)

    ax1.set_title('Toy Instance Initial Solution from PDOTWs', size=18, font='serif')
    
    return fig1, ax1

In [ ]:
truck_colors = ['b', 'r', 'g']
cargo_colors = ['#8c510a', '#bf812d', '#dfc27d', '#f6e8c3',
                '#f5f5f5','#c7eae5','#80cdc1','#35978f','#01665e']



route_sol = pdpt_rasd(dir_)



In [ ]:
from matplotlib.lines import Line2D
import matplotlib.font_manager as font_manager

legend_font = font_manager.FontProperties(family='serif',style='normal', size=12)

In [ ]:

fig1, ax1 = plot_instance(dir_, _, cargo_colors, legend_font)

fig1.savefig(dir_+'/toy.png', dpi=150, transparent=True)


In [ ]:
fig2, ax2 = plot_init_Sol(dir_, truck_colors, cargo_colors, legend_font)
fig2.savefig(dir_+'/toy_initSol.png', dpi=150, transparent=True)


In [2]:
fig3, ax3 = plot_rasd_sol(dir_, truck_colors, cargo_colors, legend_font)
fig3.savefig(dir_+'/toy_rasdSol.png', dpi=150, transparent=True)


NameError: name 'plot_rasd_sol' is not defined